In [1]:
import os 
import sys 
parent_dir = os.path.dirname(os.getcwd())
sys.path.append("/home/jbc0071/Documents/CANIDS/code")

from model.CANnoloAttackDetector import CANnoloAttackDetector
from data_helpers.CANDataset import CANDataset
from helpers import calculate_feature_vec_length, seperate_attack_loader, calculate_metrics
from dotenv import load_dotenv
# from data_helpers.CANDataLoader import CANDataLoader
import os

load_dotenv()
data_path = os.getenv('DATA_PATH')
dataset = CANDataset(data_path, log_verbosity=1)

config = {
    "batch_size": 32,
    "delta_time_last_msg": {
        "specific_to_can_id": False,
        "records_back": 30
    },
    "delta_time_last_same_aid": {
        "specific_to_can_id": True,
        "records_back": 15
    },
}

ambient_loader, validation_loader, attack_loader = dataset.get_dataloaders(config)

attack_loaders = seperate_attack_loader(attack_loader, config, remove_non_labelled=True)


Found ambient and attack directories.
Loading CAN metadata...
Parquet files found...
Found processed parquet files...
Loading processed parquet files...
Loading processing data into 'CANData' structure
No attack labels in accelerator_attack_drive_1.log
No attack labels in accelerator_attack_drive_2.log
No attack labels in accelerator_attack_reverse_1.log
No attack labels in accelerator_attack_reverse_2.log
Found attack labels in correlated_signal_attack_1.log
Found attack labels in correlated_signal_attack_2.log
Found attack labels in correlated_signal_attack_3.log
Found attack labels in fuzzing_attack_1.log
Found attack labels in fuzzing_attack_2.log
Found attack labels in fuzzing_attack_3.log
Found attack labels in max_engine_coolant_temp_attack.log
Found attack labels in max_speedometer_attack_1.log
Found attack labels in max_speedometer_attack_2.log
Found attack labels in max_speedometer_attack_3.log
Found attack labels in reverse_light_off_attack_1.log
Found attack labels in rever

In [2]:
display(attack_loaders)

In [3]:

unique_can_ids = dataset.get_unique_can_ids()
num_can_ids = len(unique_can_ids)
feature_vec_length = ambient_loader.features_len

# Load model
model_path = 'models/canolo_model_112.pt'
thresholds = [6.3e-06]

model_config = {
    "embedding_dim": num_can_ids,
    "lstm_units": 128,
    "dense_units": 256,
    "dropout_rate": 0.2,
    "num_embeddings": max(unique_can_ids) + 1, # not sure why + 1 rn but it works
    "feature_vec_length": calculate_feature_vec_length(config)
}

detector = CANnoloAttackDetector(model_path, threshold, model_config)


#     # make enought detectors the length of attack_loaders * deep copy
#     detectors = [deepcopy(detector) for _ in range(len(attack_loaders))]
#     # make a list of tuples of detectors and loaders
#     detector_loader_pairs = zip(detectors, attack_loaders)
#     # run the detectors in parallel
#     results = executor.map(lambda x: calculate_metrics(x[0].detect_attacks(x[1])), detector_loader_pairs)
#     # convert the results to a dictionary
#     results = dict(zip([loader.can_data[0].filename[0] for loader in attack_loaders], results))


NameError: name 'threshold' is not defined

In [ ]:
# detector.determine_threshold(ambient_loader)

# 0.0000065
# 0.0000065
# 0.0000065
# 0.0000063
# 0.000165
# .122

# ninetyfive_percentile = .0000156


In [ ]:

filenames = []
for loader in attack_loaders:
    # print the column names
    filenames = loader.can_data[0].filename[0]

In [4]:
from concurrent.futures import ProcessPoolExecutor
from detect import detect, detect_wrapper
import time

# def detect(loader, detector):
#     filename = loader.can_data[0].filename[0]
#     print(f"Testing on {filename}")
#     return filename, calculate_metrics(detector.detect_attacks(loader))

# def detect_wrapper(args):
#     return detect(args[0], args[1])

thresholds = [.000095, .000085, .000075, .000065, .000055]

num_attack_files = len(attack_loaders)
detectors = []

start_time = time.time()
for _ in range(num_attack_files):
    detector = CANnoloAttackDetector(model_path, thresholds, model_config, force_cpu=True)
    detectors.append(detector)

with ProcessPoolExecutor() as executor:
    list_of_filename_and_results = list(executor.map(detect_wrapper, zip(attack_loaders, detectors)))

for loader in attack_loaders:
    loader.reset()
end_time = time.time()

print(f"Finished in {end_time - start_time} seconds")

print(list_of_filename_and_results)




Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Testing on correlated_signal_attack_1.log
Testing on correlated_signal_attack_3.log
Testing on correlated_signal_attack_2.log
Testing on fuzzing_attack_1.log
Testing on fuzzing_attack_2.logTesting on fuzzing_attack_3.log

Testing on max_engine_coolant_temp_attack.log
Testing on max_speedometer_attack_1.log
Testing on max_speedometer_attack_2.log
Testing on reverse_light_off_attack_1.logTesting on max_speedometer_attack_3.log

Testing on reverse_light_off_attack_2.log
Testing on reverse_light_off_attack_3.log
Testing on reverse_light_on_attack_3.log
Testing on reverse_light_on_attack_1.log
Testing on reverse_light_on_attack_2.log
Finished in 172.68796992301

In [5]:
display(list_of_filename_and_results)

[('correlated_signal_attack_1.log',
  {9.5e-05: (0.9597595548089038, 0.0, 0.0, 0, [[0, 980], [2086, 73126]]),
   8.5e-05: (0.9597595548089038, 0.0, 0.0, 0, [[0, 980], [2086, 73126]]),
   7.5e-05: (0.9597595548089038, 0.0, 0.0, 0, [[0, 980], [2086, 73126]]),
   6.5e-05: (0.9597595548089038, 0.0, 0.0, 0, [[0, 980], [2086, 73126]]),
   5.5e-05: (0.9597595548089038, 0.0, 0.0, 0, [[0, 980], [2086, 73126]])}),
 ('correlated_signal_attack_2.log',
  {9.5e-05: (0.9562928501469148, 0.0, 0.0, 0, [[0, 717], [2139, 62488]]),
   8.5e-05: (0.9562928501469148, 0.0, 0.0, 0, [[0, 717], [2139, 62488]]),
   7.5e-05: (0.9562928501469148, 0.0, 0.0, 0, [[0, 717], [2139, 62488]]),
   6.5e-05: (0.9562622428991185, 0.0, 0.0, 0, [[0, 719], [2139, 62486]]),
   5.5e-05: (0.9562469392752204, 0.0, 0.0, 0, [[0, 720], [2139, 62485]])}),
 ('correlated_signal_attack_3.log',
  {9.5e-05: (0.9578316326530613, 0.0, 0.0, 0, [[0, 390], [1263, 37547]]),
   8.5e-05: (0.9578061224489796, 0.0, 0.0, 0, [[0, 391], [1263, 37546]]),


In [ ]:
print(meta_data)

In [ ]:
{
    .000095: [
        ('correlated_signal_attack_1.log', (0.9597595548089038, 0.0, 0.0, 0, [[0, 980], [2086, 73126]])),
        ('correlated_signal_attack_2.log', (0.9562928501469148, 0.0, 0.0, 0, [[0, 717], [2139, 62488]])), 
        ('correlated_signal_attack_3.log', (0.9578316326530613, 0.0, 0.0, 0, [[0, 390], [1263, 37547]])), 
        ('fuzzing_attack_1.log', (0.9910745614035088, 0.002680965147453083, 0.027777777777777776, 0.004889975550122249, [[1, 372], [35, 45192]])), 
        ('fuzzing_attack_2.log', (0.97625, 0.004322766570605188, 0.2, 0.00846262341325811, [[3, 691], [12, 28894]])), 
        ('fuzzing_attack_3.log', (0.9901666666666666, 0.008547008547008548, 0.3333333333333333, 0.016666666666666666, [[1, 116], [2, 11881]])), 
        ('max_engine_coolant_temp_attack.log', (0.9894626103752759, 0.0, 0.0, 0, [[0, 569], [42, 57373]])), 
        ('max_speedometer_attack_1.log', (0.9796565099168266, 0.0, 0.0, 0, [[0, 1626], [2444, 195994]])), 
        ('max_speedometer_attack_2.log', (0.9710740177736202, 0.0, 0.0, 0, [[0, 818], [3140, 132874]])), 
        ('max_speedometer_attack_3.log', (0.9591699265527702, 0.0, 0.0, 0, [[0, 2076], [6107, 192233]])), 
        ('reverse_light_off_attack_1.log', (0.9784959758551308, 0.0, 0.0, 0, [[0, 696], [672, 62248]])),
        ('reverse_light_off_attack_2.log', (0.9631100171232877, 0.0, 0.0, 0, [[0, 1076], [2371, 89993]])), 
        ('reverse_light_off_attack_3.log', (0.9707338866004361, 0.0, 0.0, 0, [[0, 1431], [2434, 128199]])), 
        ('reverse_light_on_attack_1.log', (0.9769328080963362, 0.0, 0.0, 0, [[0, 890], [1991, 122015]])), 
        ('reverse_light_on_attack_2.log', (0.966156437851464, 0.0, 0.0, 0, [[0, 1896], [3689, 159439]])), 
        ('reverse_light_on_attack_3.log', (0.972242727471566, 0.0, 0.0, 0, [[0, 1710], [2351, 142243]]))
        ],
    .000085: data..
}

# structure:
# (filename, (accuracy, precision, recall, f1_score, confusion_matrix)
# ex:
# ('reverse_light_on_attack_3.log', (0.972242727471566, 0.0, 0.0, 0, [[0, 1710], [2351, 142243]]))

# graph thresholds vs avg f1 score for that group


# for the best performing threshold make a graph for each filename and show the results




In [ ]:
detector.threshold = 5.2644566676462956e-05


In [4]:
from tqdm import tqdm

meta_results = {}
for threshold in [.000095, .000085, .000075, .000065, .000055]:
    results = {}
    for i, loader in enumerate(attack_loaders):
        filename = loader.can_data[0].filename[0]
        print(f"Testing on {filename}")
        results[filename] = calculate_metrics(detector.detect_attacks(loader))
        print(results[filename])
    meta_results[threshold] = results

print(meta_results)

NameError: name 'attack_loaders' is not defined